In [1]:
%pip install "numpy<2.0"

%pip install --upgrade pandas
%pip install --upgrade transformers
%pip install --upgrade datasets
%pip install --upgrade torch
%pip install --upgrade huggingface_hub

import numpy as np

  Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-

In [8]:
import torch

from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW

model_name='google-bert/bert-base-uncased'

model=BertForSequenceClassification.from_pretrained(model_name,num_labels=3) #3 labels for +ve,-ve and neutral sentiment of text

tokeniser=BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
def sentiment_analysis(text):
  input_text=tokeniser(text,return_tensors="pt",padding=True,truncation=True,max_length=512)
  pred=model(**input_text)
  pred_class=torch.argmax(pred.logits,dim=1)
  class_lables={0:"negative",1:"neutral",2:"positive"}

  sentiment=class_lables[pred_class.item()]
  return sentiment

print("The sentiment for 'how on earth can I analyse this?' is:",sentiment_analysis("how on earth can I analyse this?"))


The sentiment for 'how on earth can I analyse this?' is: negative


In [20]:
batch_tweets = [
    "The world is going through madness",
    "Technology is evolving faster than ever",
    "Small steps every day lead to big changes",
    "Positivity is contagious, spread it wisely",
    "AI is shaping the future in unexpected ways",
    "Take a break, your mind deserves it",
    "Opportunities often hide behind challenges",
    "Kindness never goes out of style",
    "Learning never stops, keep growing",
    "Sometimes doing nothing is also progress",
    "Life is good"
]

tweets=[]

for tweet in batch_tweets:
  tweets.append(sentiment_analysis(tweet))
tweets



['negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative']

In [21]:
dataset_name="mteb/tweet_sentiment_extraction"

import pandas as pd
from datasets import load_dataset
tweet_dataset=load_dataset(dataset_name)

print(tweet_dataset)


df=pd.DataFrame(tweet_dataset["train"])
df.head(7)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 26732
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3432
    })
})


,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,1,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,2,positive


In [23]:
model_name='google-bert/bert-base-uncased'
from transformers import BertTokenizer
tokeniser=BertTokenizer.from_pretrained(model_name)

def tokenise_them(examples):
  return tokeniser(examples["text"],padding="max_length",truncation=True,max_length=512)

tokenised_datasets=tweet_dataset.map(tokenise_them,batched=True)


training_dataset=tokenised_datasets["train"].shuffle(seed=124).select(range(500))
evaluation_dataset=tokenised_datasets["test"].shuffle(seed=124).select(range(500))

df=pd.DataFrame(evaluation_dataset)
df.head(7)



Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

,id,text,label,label_text,input_ids,token_type_ids,attention_mask
0,0a5e7d6cb1,I so know what you mean,1,neutral,"[101, 1045, 2061, 2113, 2054, 2017, 2812, 102,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,da15c4b003,_jane hope yours will go away quicker than m...,2,positive,"[101, 1035, 4869, 3246, 6737, 2097, 2175, 2185...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,28db608c0d,Gonna go to work with my honey today!!!! So ex...,2,positive,"[101, 6069, 2175, 2000, 2147, 2007, 2026, 6861...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,0ce20ef06e,6 am. you?,1,neutral,"[101, 1020, 2572, 1012, 2017, 1029, 102, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,afe5ce5d7d,"Going to group therapy session, can`t wait to ...",1,neutral,"[101, 2183, 2000, 2177, 7242, 5219, 1010, 2064...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,c4df4b1e33,Now I`m all sad because I`ll probably never se...,0,negative,"[101, 2085, 1045, 1036, 1049, 2035, 6517, 2138...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,0e3d04ed05,"I am in Weirton Julie, way up North at the ti...",0,negative,"[101, 1045, 2572, 1999, 16658, 2669, 7628, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [24]:
import torch

from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW

model_name='google-bert/bert-base-uncased'


def sentiment_analysis(text):
  input_text=tokeniser(text,return_tensors="pt",padding=True,truncation=True,max_length=512)
  pred=model(**input_text)
  pred_class=torch.argmax(pred.logits,dim=1)
  class_lables={0:"negative",1:"neutral",2:"positive"}

  sentiment=class_lables[pred_class.item()]
  return sentiment

print("The sentiment for 'how on earth can I analyse this?' is:",sentiment_analysis("how on earth can I analyse this?"))


The sentiment for 'how on earth can I analyse this?' is: negative


In [32]:
from transformers import Trainer,TrainingArguments


training_args=TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    no_cuda=True
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [36]:
!pip install --upgrade transformers

In [37]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='ModelOutput',
    eval_strategy="epoch",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    no_cuda=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=evaluation_dataset,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ksparsh443 (ksparsh443-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.052077


TrainOutput(global_step=63, training_loss=1.082111540294829, metrics={'train_runtime': 4398.7132, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.014, 'total_flos': 131556708864000.0, 'train_loss': 1.082111540294829, 'epoch': 1.0})

In [38]:
trainer.save_model("fine-tuned-Bert")
tokeniser.save_pretrained("fine-tuned-Bert")

model=BertForSequenceClassification.from_pretrained("fine-tuned-Bert")
tokeniser=BertTokenizer.from_pretrained("fine-tuned-Bert")

def sentiment_analysis(text):
  input_text=tokeniser(text,return_tensors="pt",padding=True,truncation=True,max_length=512)
  pred=model(**input_text)
  pred_class=torch.argmax(pred.logits,dim=1)
  class_lables={0:"negative",1:"neutral",2:"positive"}

  sentiment=class_lables[pred_class.item()]
  return sentiment

print("The sentiment for 'how on earth can I analyse this?' is:",sentiment_analysis("how on earth can I analyse this?"))


The sentiment for 'how on earth can I analyse this?' is: neutral


In [39]:
batch_tweets = [
    "The world is going through madness",
    "Technology is evolving faster than ever",
    "Small steps every day lead to big changes",
    "Positivity is contagious, spread it wisely",
    "AI is shaping the future in unexpected ways",
    "Take a break, your mind deserves it",
    "Opportunities often hide behind challenges",
    "Kindness never goes out of style",
    "Learning never stops, keep growing",
    "Sometimes doing nothing is also progress",
    "Life is good"
]

tweets=[]

for tweet in batch_tweets:
  tweets.append(sentiment_analysis(tweet))
tweets



['neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']